Load data

In [57]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'colab/'

Mounted at /content/gdrive


In [58]:
# download streetstyle data
!wget https://s3.amazonaws.com/kmatzen/streetstyle27k.tar

--2019-05-07 22:42:26--  https://s3.amazonaws.com/kmatzen/streetstyle27k.tar
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.237.37
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.237.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2464604160 (2.3G) [application/x-tar]
Saving to: ‘streetstyle27k.tar.1’

streetstyle27k.tar.  84%[===============>    ]   1.95G  59.3MB/s    eta 13s    ^C


In [0]:
# Extracting zip file
!tar xvf streetstyle27k.tar

Output hidden; open in https://colab.research.google.com to view.

In [0]:
# get the attribution files
!wget https://s3.amazonaws.com/kmatzen/streetstyle27k.manifest

--2019-05-07 22:12:46--  https://s3.amazonaws.com/kmatzen/streetstyle27k.manifest
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.65.107
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.65.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6843031 (6.5M) [text/plain]
Saving to: ‘streetstyle27k.manifest’

streetstyle27k.mani 100%[===================>]   6.53M  27.4MB/s    in 0.2s    

2019-05-07 22:12:46 (27.4 MB/s) - ‘streetstyle27k.manifest’ saved [6843031/6843031]



In [63]:
#!mkdir data
#!mv streetstyle27k data/images
#!mv streetstyle27k.manifest data/streetstyle27k.manifest

#%cd ./data/images
!ls -l
#%cd ../
#!rm streetstyle27k.tar.1

total 2406856
drwxr-xr-x 3 root root       4096 May  7 22:23 data
drwx------ 3 root root       4096 May  7 22:42 gdrive
drwxr-xr-x 1 root root       4096 May  3 16:44 sample_data
-rw-r--r-- 1 root root 2464604160 Sep  4  2017 streetstyle27k.tar


In [66]:
!ls -l

total 2406856
drwxr-xr-x 3 root root       4096 May  7 22:23 data
drwx------ 3 root root       4096 May  7 22:42 gdrive
drwxr-xr-x 1 root root       4096 May  3 16:44 sample_data
-rw-r--r-- 1 root root 2464604160 Sep  4  2017 streetstyle27k.tar


Create a database to store images and attributes

In [0]:
import sqlite3
import os

def fetch_entry(line):
    split = line.split(',')
    if len(split) == 25:
        # primary key is integer
        split[0] = int(split[0])
        # remove the newline
        split[-1] = split[-1][:-1]
        # generate a local url
        split[1] = '/data/streetstyle27k/' + split[1][33] + '/' + split[1][34] + '/' + split[1][35] + '/' + split[1][33:]
        return tuple(split)
    else:
        return None

def store_to_db(manifest_file, db_name):
    # open sqlite3 database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    # build tables from schema
    cursor.executescript(database_schema)
    # fill the tables
    with open(manifest_file) as fid:
        # read first line as header, ignore
        fid.readline()
        # create a tuple entry for each line in manifest
        for line in fid:
            entry = fetch_entry(line)
            if entry is not None:
                cursor.execute("INSERT OR REPLACE INTO streetstyle27k VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                                entry)
    conn.commit()
    conn.close()
    return
database_schema = """
                        CREATE TABLE IF NOT EXISTS streetstyle27k (
                                    id integer PRIMARY KEY,
                                    url TEXT,
                                    created_time TEXT,
                                    city_id TEXT,
                                    month_id TEXT,
                                    lat TEXT,
                                    lon TEXT,
                                    x1 TEXT,
                                    y1 TEXT,
                                    x2 TEXT,
                                    y2 TEXT,
                                    width TEXT,
                                    height TEXT,
                                    clothing_pattern TEXT,
                                    major_color TEXT,
                                    wearing_necktie TEXT,
                                    collar_presence TEXT,
                                    wearing_scarf TEXT,
                                    sleeve_length TEXT,
                                    neckline_shape TEXT,
                                    clothing_category TEXT,
                                    wearing_jacket TEXT,
                                    wearing_hat TEXT,
                                    wearing_glasses TEXT,
                                    multiple_layers TEXT
                                );
"""

manifest_file = 'data/streetstyle27k.manifest'
db_name = 'data/streetstyle27k.db'
store_to_db(manifest_file, db_name)

In [56]:
import matplotlib.pyplot as plt
from PIL import Image
import glob
count=0
fig=plt.figure(figsize=(16, 9))
filename = 'data/images/d/f/f/dff8b1017226167906804e492526d9e6_726861536941066133_632544368.jpg'
img=Image.open(filename)
plt.imshow(img)


FileNotFoundError: ignored

<Figure size 1152x648 with 0 Axes>